## **Preprocessing dát o VŠ učiteľoch**

In [1]:
import pandas as pd

In [67]:
public_fulltime_22 = pd.read_excel('../data/cvtisr/vs2022/vs2022.xls', sheet_name='Tab16v')
display(public_fulltime_22)

,Vysoké školy k 31. 10. 2022 - verejné,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,138,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,Škola,NaN,Učitelia na ustanovený pracovný čas (plný úväzok),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,spolu,z toho,profesori,NaN,NaN,docenti,NaN,NaN,odborní asistenti,NaN,asis-,lek-,hosťu-
2,Fakulta,NaN,NaN,NaN,spolu,v tom,NaN,spolu,v tom,NaN,spolu,v tom,NaN,NaN,júci
3,NaN,NaN,NaN,ženy,NaN,DrSc.,"CSc.,",NaN,DrSc.,"CSc.,",NaN,"CSc.,",tenti,tori,profe-
4,NaN,NaN,NaN,NaN,NaN,NaN,PhD.,NaN,NaN,PhD.,NaN,PhD.,NaN,NaN,sori
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,NaN,Fa dramat. umení AU,35,10,6,1,3,18,0,14,11,10,0,0,0
139,NaN,Fa múzických umení AU,66,23,12,0,8,16,0,14,32,21,2,4,0
140,NaN,Fa výtvarných umení AU,33,9,5,0,3,11,0,7,17,11,0,0,0
141,AKADÉMIA UMENÍ,NaN,134,42,23,1,14,45,0,35,60,42,2,4,0


In [68]:
public_fulltime_22_summary = pd.read_excel('../data/cvtisr/vs2022/vs2022.xls', sheet_name='Tab16v', header=5,
                                          usecols="A:D").rename(columns={'Unnamed: 0': 'univerzita', 'Unnamed: 1': 'fakulta', 'Unnamed: 2': 'spolu full-time', 'Unnamed: 3': 'zeny full-time'})
display(public_fulltime_22_summary)

,univerzita,fakulta,spolu full-time,zeny full-time
0,NaN,Rímskokat.bohosl.fak.UK,21,2
1,NaN,Evanj. bohosl. fak. UK,14,2
2,NaN,Fa matem.fyz. a inf. UK,211,63
3,NaN,Prírodovedecká fak. UK,319,139
4,NaN,Fakulta managementu UK,69,39
...,...,...,...,...
133,NaN,Fa dramat. umení AU,35,10
134,NaN,Fa múzických umení AU,66,23
135,NaN,Fa výtvarných umení AU,33,9
136,AKADÉMIA UMENÍ,NaN,134,42


In [69]:
# backfill / bfill: replaces NaN values with the next non-NaN value that appears in the column
public_fulltime_22_summary['univerzita'] = public_fulltime_22_summary['univerzita'].fillna(method='backfill')
display(public_fulltime_22_summary)

,univerzita,fakulta,spolu full-time,zeny full-time
0,UNIVERZITA KOMENSKÉHO,Rímskokat.bohosl.fak.UK,21,2
1,UNIVERZITA KOMENSKÉHO,Evanj. bohosl. fak. UK,14,2
2,UNIVERZITA KOMENSKÉHO,Fa matem.fyz. a inf. UK,211,63
3,UNIVERZITA KOMENSKÉHO,Prírodovedecká fak. UK,319,139
4,UNIVERZITA KOMENSKÉHO,Fakulta managementu UK,69,39
...,...,...,...,...
133,AKADÉMIA UMENÍ,Fa dramat. umení AU,35,10
134,AKADÉMIA UMENÍ,Fa múzických umení AU,66,23
135,AKADÉMIA UMENÍ,Fa výtvarných umení AU,33,9
136,AKADÉMIA UMENÍ,NaN,134,42


In [70]:
# remove the summary rows: 
public_fulltime_22_summary = public_fulltime_22_summary.dropna(axis=0, subset='fakulta').reset_index(drop=True)

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.precision', 3,):
    print(public_fulltime_22_summary)

                  univerzita                  fakulta  spolu full-time  \
0      UNIVERZITA KOMENSKÉHO  Rímskokat.bohosl.fak.UK               21   
1      UNIVERZITA KOMENSKÉHO   Evanj. bohosl. fak. UK               14   
2      UNIVERZITA KOMENSKÉHO  Fa matem.fyz. a inf. UK              211   
3      UNIVERZITA KOMENSKÉHO   Prírodovedecká fak. UK              319   
4      UNIVERZITA KOMENSKÉHO   Fakulta managementu UK               69   
5      UNIVERZITA KOMENSKÉHO      Lekárska fakulta UK              505   
6      UNIVERZITA KOMENSKÉHO  Jesseniova lek. fak. UK              184   
7      UNIVERZITA KOMENSKÉHO    Farmaceutická fak. UK              107   
8      UNIVERZITA KOMENSKÉHO     Právnická fakulta UK              136   
9      UNIVERZITA KOMENSKÉHO   Filozofická fakulta UK              262   
10     UNIVERZITA KOMENSKÉHO  Fa soc.a ekonom.vied UK               50   
11     UNIVERZITA KOMENSKÉHO   Pedagogická fakulta UK              140   
12     UNIVERZITA KOMENSKÉHO   Fakulta

In [73]:
# pridame stlpec verejne
public_fulltime_22_summary['typ'] = 'verejná'
public_fulltime_22_summary = public_fulltime_22_summary[['univerzita', 'fakulta', 'typ', 'spolu full-time', 'zeny full-time']]
display(public_fulltime_22_summary)

,univerzita,fakulta,typ,spolu full-time,zeny full-time
0,UNIVERZITA KOMENSKÉHO,Rímskokat.bohosl.fak.UK,verejná,21,2
1,UNIVERZITA KOMENSKÉHO,Evanj. bohosl. fak. UK,verejná,14,2
2,UNIVERZITA KOMENSKÉHO,Fa matem.fyz. a inf. UK,verejná,211,63
3,UNIVERZITA KOMENSKÉHO,Prírodovedecká fak. UK,verejná,319,139
4,UNIVERZITA KOMENSKÉHO,Fakulta managementu UK,verejná,69,39
...,...,...,...,...,...
112,VŠ MÚZICKÝCH UMENÍ,VŠMU - rektorát,verejná,3,3
113,VŠ VÝTVARNÝCH UMENÍ,Fa výtvar. umení VŠVU,verejná,118,40
114,AKADÉMIA UMENÍ,Fa dramat. umení AU,verejná,35,10
115,AKADÉMIA UMENÍ,Fa múzických umení AU,verejná,66,23


- Definujme si na to funkciu:

In [74]:
def fix_table(df, university_type):
    """
    Fill the university names. Remove summary rows. Add type of university.
    """
    # backfill / bfill: replaces NaN values with the next non-NaN value that appears in the column
    df['univerzita'] = df['univerzita'].fillna(method='backfill')
    # remove summary rows = rows without specified Faculty
    df = df.dropna(axis=0, subset='fakulta').reset_index(drop=True)
    df['typ'] = university_type
    return df

In [86]:
public_parttime_22_summary = pd.read_excel('../data/cvtisr/vs2022/vs2022.xls', sheet_name='Tab17v', header=5,
                                          usecols="A:D").rename(columns={'Unnamed: 0': 'univerzita', 'Unnamed: 1': 'fakulta', 'Unnamed: 2': 'spolu part-time', 'Unnamed: 3': 'zeny part-time'})
public_parttime_22_summary = fix_table(public_parttime_22, university_type='verejná')
display(public_parttime_22_summary)

,univerzita,fakulta,spolu part-time,zeny part-time,typ
0,UNIVERZITA KOMENSKÉHO,Rímskokat.bohosl.fak.UK,19,3,verejná
1,UNIVERZITA KOMENSKÉHO,Evanj. bohosl. fak. UK,2,2,verejná
2,UNIVERZITA KOMENSKÉHO,Fa matem.fyz. a inf. UK,21,5,verejná
3,UNIVERZITA KOMENSKÉHO,Prírodovedecká fak. UK,48,20,verejná
4,UNIVERZITA KOMENSKÉHO,Fakulta managementu UK,2,1,verejná
...,...,...,...,...,...
93,VŠ MÚZICKÝCH UMENÍ,VŠMU - rektorát,2,2,verejná
94,VŠ VÝTVARNÝCH UMENÍ,Fa výtvar. umení VŠVU,12,4,verejná
95,AKADÉMIA UMENÍ,Fa dramat. umení AU,2,1,verejná
96,AKADÉMIA UMENÍ,Fa múzických umení AU,8,4,verejná


In [87]:
public_22 = pd.merge(public_fulltime_22_summary, public_parttime_22_summary, how="outer", on=['univerzita', 'fakulta', 'typ'])
display(public_22)

,univerzita,fakulta,typ,spolu full-time,zeny full-time,spolu part-time,zeny part-time
0,UNIVERZITA KOMENSKÉHO,Rímskokat.bohosl.fak.UK,verejná,21,2,19.0,3.0
1,UNIVERZITA KOMENSKÉHO,Evanj. bohosl. fak. UK,verejná,14,2,2.0,2.0
2,UNIVERZITA KOMENSKÉHO,Fa matem.fyz. a inf. UK,verejná,211,63,21.0,5.0
3,UNIVERZITA KOMENSKÉHO,Prírodovedecká fak. UK,verejná,319,139,48.0,20.0
4,UNIVERZITA KOMENSKÉHO,Fakulta managementu UK,verejná,69,39,2.0,1.0
...,...,...,...,...,...,...,...
112,VŠ MÚZICKÝCH UMENÍ,VŠMU - rektorát,verejná,3,3,2.0,2.0
113,VŠ VÝTVARNÝCH UMENÍ,Fa výtvar. umení VŠVU,verejná,118,40,12.0,4.0
114,AKADÉMIA UMENÍ,Fa dramat. umení AU,verejná,35,10,2.0,1.0
115,AKADÉMIA UMENÍ,Fa múzických umení AU,verejná,66,23,8.0,4.0
